# Reimplementing the Transformer to learn more about the Attention mechanism
## Baptiste Amato, Alexis Durocher, Gabriel Hurtado, Alexandre Jouandin, Vincent Marois

Spring 2019 CS 7643 Deep Learning Class Project
Georgia Tech

# TEMPLATE HERE: 
https://www.cc.gatech.edu/classes/AY2019/cs7643_spring/assets/project_webpage_template/

# Introduction / Background / Motivation

(5 points) What did you try to do? What problem did you try to solve? Articulate your objectives using absolutely no jargon.
#### The goal of this project is to understand the Transformer model, reproduce the paper's results and try to provide some food for thoughts on what to improve.

(5 points) How is it done today, and what are the limits of current practice?
#### The training mentioned in the paper was done on several GPU (?give their specs?).

(5 points) Who cares? If you are successful, what difference will it make?
#### Improvements on the Transformer model, and more generally the Attention mechanism, would benefit the entire research community that works on Natural Language Processing, both for theoretical and applied research.
#### We managed to get close to the original paper's results with limited hardware and time. This project then proves the accessibility of such a complex model (?to finish?)

# Approach

(10 points) What did you do exactly? How did you solve the problem? Why did you think it would be successful? Is anything new in your approach?
#### After implementing the entire model, we made sure every building block was properly connected to the others by trying a simple task: copy. Indeed, a working implementation a such a heavy model should have no problem copying inputs to outputs.
#### That being done, we thought about how we could reproduce the original paper's results without having access to the computational power the authors had. When plotting the distribution of the dataset's sequences length (?show distribution curve?), we observe that 90% of the sequences has a length of maximum 40 (the largest sequence having a length of 102). This helped making the training faster without harming the results too much.
#### (?complete?)

(5 points) What problems did you anticipate? What problems did you encounter? Did the very first thing you tried work?
#### (?talk about the difference of workflow between training and testing?)


# Experiments and Results

(10 points) How did you measure success? What experiments were used? What were the results, both quantitative and qualitative? Did you succeed? Did you fail? Why?

#### On a quantitative point of view, the loss (for both training and validation) is a good indicator of how well the model does; we used KL-divergence.
#### (?show Tensorboard plots?)
#### (?show visuals from poster?)

# In addition, 20 more points will be distributed based on presentation quality and Deep Learning knowledge

(5 points) Appropriate use of visual aids. Are the ideas presented with appropriate illustration? Is the problem effectively visualized? Is the approach visualized appropriately? Are the results presented clearly; are the important differences illustrated? Every section and idea does not need a visual aid, but the most interesting and complex parts of the project should be illustrated.

(5 points) Overall clarity. Is the presentation clear? Can a peer who has also taken Deep Learning understand all of the points addressed above? Is sufficient detail provided?

(10 points) Finally, points will be distributed based on your understanding of how your project relates to Deep Learning. Here are some questions to think about:

- What was the structure of your problem? How did the structure of your model reflect the structure of your problem?
- What parts of your model had learned parameters (e.g., convolution layers) and what parts did not (e.g., post-processing classifier probabilities into decisions)?
- What representations of input and output did the neural network expect? How was the data pre/post-processed?
What was the loss function?
- Did the model overfit? How well did the approach generalize?
- What hyperparameters did the model have? How were they chosen? How did they affect performance? What optimizer was used?
- What Deep Learning framework did you use?
- What existing code or models did you start with and what did those starting points provide?
